In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType, AgentExecutor, create_openai_functions_agent
#from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.google_search import GoogleSearchResults
import warnings
warnings.filterwarnings(action='ignore')
load_dotenv()

True

In [57]:
llm = AzureChatOpenAI(
    deployment_name='gpt-4-turbo',
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
    temperature=0
) 

In [31]:
template = "What is ISIN of {company}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
chain.invoke({"company": "Beasley Broadcast Group, Inc., Class A"})

AIMessage(content="The International Securities Identification Number (ISIN) is a unique code that identifies a specific securities issue. For Beasley Broadcast Group, Inc., Class A, you would need to look up the current ISIN on a financial database or contact the company directly, as this information can change and may not be readily available without access to a financial database or the company's investor relations department.\n\nAs of my last update, I do not have real-time access to financial databases to provide the current ISIN for Beasley Broadcast Group, Inc., Class A. Please check the latest financial information from a reliable source or the company's official investor relations page for the most accurate and up-to-date ISIN.")

In [4]:
template = "What is CUSIP of {company}"
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
chain.invoke({"company": "Beasley Broadcast Group, Inc., Class A"})

AIMessage(content='The CUSIP number for Beasley Broadcast Group, Inc., Class A is 074014101.')

In [5]:
%%time
template = """Extract coupon and maturity date from {company}. Return in markdown table format i.e. Company, Coupon, Maturity Date
e.g.  COCA-COLA EURO 0.8% 21-03/05/2024, Company is COCA-COLA, Coupon is 0.8%, Maturity Date is 03/05/2024
"""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
print(chain.invoke({"company": "LLOYDS BANK PLC 4.5% 14-04/11/2024"}).content)

| Company       | Coupon | Maturity Date |
| ------------- | ------ | ------------- |
| LLOYDS BANK PLC | 4.5%   | 04/11/2024    |
CPU times: total: 31.2 ms
Wall time: 1.49 s


In [6]:
template = """ What is CUSIP and ISIN of {company}. Return the result in markdown table format. 
If you are not sure about a certain security return NA."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
result = chain.invoke({"company": """1ST SOURCE CORP  COM                
ACUITY BRANDS INC  COM              
AMERICOLD REALTY TRUST INC  COM
APPLIED INDUSTRIAL TECHNOLOGIES  COM
APTARGROUP INC  COM                 
ATRION CORP  COM                    
BADGER METER INC  COM               
CABLE ONE INC  COM                  
CACTUS INC  CL A
CAMDEN NATIONAL CORP  COM           """})
print(result.content)

| Company Name | CUSIP | ISIN |
| --- | --- | --- |
| 1ST SOURCE CORP COM | 336901103 | US3369011032 |
| ACUITY BRANDS INC COM | 00508Y102 | US00508Y1029 |
| AMERICOLD REALTY TRUST INC COM | 03064D108 | US03064D1081 |
| APPLIED INDUSTRIAL TECHNOLOGIES COM | 03820C105 | US03820C1053 |
| APTARGROUP INC COM | 038336103 | US0383361039 |
| ATRION CORP COM | 049904105 | US0499041053 |
| BADGER METER INC COM | 056525108 | US0565251081 |
| CABLE ONE INC COM | 12685J105 | US12685J1051 |
| CACTUS INC CL A | 14888D208 | US14888D2089 |
| CAMDEN NATIONAL CORP COM | 133034108 | US1330341081 |

Please note that the CUSIP and ISIN numbers are subject to change and should be verified for accuracy.


In [32]:
%%time
template = """You are an expert in securities identification, I will be providing you a list of bonds with maturity dates. 
Get the ISIN for each bond and return the output in markdown table format.
{company}"""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
result = chain.invoke({"company": """
BONOS Y OBLIG DEL ESTADO 30/04/2029
BONOS Y OBLIG DEL ESTADO 30/04/2031
BONOS Y OBLIG DEL ESTADO 30/04/2032
BONOS Y OBLIG DEL ESTADO 30/07/2033
BONOS Y OBLIG DEL ESTADO 30/07/2035
"""})
print(result.content)

As an AI language model, I don't have real-time access to databases or external systems to look up current ISINs for securities. ISINs (International Securities Identification Numbers) are unique codes assigned to securities to facilitate their identification. To find the ISIN for a specific bond, you would typically use financial databases, securities exchanges, or official publications from the issuing entity.

However, I can guide you on how to find the ISINs:

1. **Financial Databases**: Bloomberg, Reuters, and other financial data providers allow users to search for securities by name, issuer, maturity date, and other criteria to find the ISIN.

2. **National Numbering Agencies**: Each country has a numbering agency responsible for assigning ISINs. For Spanish bonds ("BONOS Y OBLIG DEL ESTADO"), the responsible agency is the Comisión Nacional del Mercado de Valores (CNMV). Their website or databases might have the information.

3. **Securities Exchanges**: The exchange where the b

In [7]:
%%time
template = """ What is the ISIN of {company}. Return the result in markdown table format."""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm
result = chain.invoke({"company": """WALT DISNEY CO 1.75%24"""})
print(result.content)

I'm sorry, but I couldn't find the specific ISIN for WALT DISNEY CO 1.75%24. ISINs are specific to each security and can change based on the type of security (stock, bond, etc.) and the country of issue. You may need to contact a financial advisor or use a financial database to get this information.
CPU times: total: 0 ns
Wall time: 2.25 s


In [10]:
os.environ["GOOGLE_CSE_ID"] = os.environ["GOOGLE_CSE_ID"]
os.environ["GOOGLE_API_KEY"] = os.environ["GOOGLE_API_KEY"]

In [39]:
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper(k=2)

tool = Tool(
    name="Google Search",
    description="Search Google for recent results.",
    func=search.run,
)

In [40]:
#search.run("what is the weather in Hyderabad")

In [58]:
url = LLMChain(llm=llm, prompt=prompt)

In [59]:
tools = [Tool.from_function(func = search.run, name="URL", description="Useful to find url for companies")]

In [60]:
agent = initialize_agent(tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=False)

In [61]:
agent.invoke({"input": "Who is the CEO of Microsoft"})

{'input': 'Who is the CEO of Microsoft',
 'output': 'Satya Nadella is the CEO of Microsoft.'}

In [44]:
%%time
result = agent.invoke({"input": """ You are an expert in securities identification, I will be providing you a list of bonds with maturity dates. 
Get the ISIN for each bond and return the output in markdown table format.
BONOS Y OBLIG DEL ESTADO 30/04/2029
BONOS Y OBLIG DEL ESTADO 30/04/2031
BONOS Y OBLIG DEL ESTADO 30/04/2032
BONOS Y OBLIG DEL ESTADO 30/07/2033
BONOS Y OBLIG DEL ESTADO 30/07/2035"""})
print(result["output"])

| Bond Description                     | Maturity Date | ISIN         |
|--------------------------------------|---------------|--------------|
| BONOS Y OBLIG DEL ESTADO 30/04/2029  | 30/04/2029    | ES0000012M51 |
| BONOS Y OBLIG DEL ESTADO 30/04/2031  | 30/04/2031    | ES0000012I32 |
| BONOS Y OBLIG DEL ESTADO 30/04/2032  | 30/04/2032    | ES0000012411 |
| BONOS Y OBLIG DEL ESTADO 30/07/2033  | 30/07/2033    | ES0000012L78 |
| BONOS Y OBLIG DEL ESTADO 30/07/2035  | 30/07/2035    | ES0000012932 |
CPU times: total: 562 ms
Wall time: 3min 42s


In [50]:
%%time
result = agent.invoke({"input": """ You are an expert in securities identification, I will be providing you a list of bonds with maturity dates. 
Get the ISIN for each bond and return the output in markdown table format.
BONOS Y OBLIG DEL ESTADO 30/04/2029
BONOS Y OBLIG DEL ESTADO 30/04/2031
BONOS Y OBLIG DEL ESTADO 30/04/2032
BONOS Y OBLIG DEL ESTADO 30/07/2033
BONOS Y OBLIG DEL ESTADO 30/07/2035"""})
print(result["output"])

I'm sorry, but as an AI, I don't have the capability to perform web searches or access external databases to find the ISIN for these bonds.
CPU times: total: 15.6 ms
Wall time: 2.98 s
